## Item Collaborative Filtering Pipeline

In [1]:
import os
import gc
import heapq
import pickle
import numba as nb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import math


In [2]:
%%time
df = pd.read_csv("../../allData/validationData/train_meta_data.csv")
df_test = pd.read_csv("../../allData/validationData/test_meta_data.csv")
df = pd.concat([df, df_test]).reset_index(drop = True)
npz = np.load("../../allData/validationData/train_core_data.npz")
npz_test = np.load("../../allData/validationData/test_core_data.npz")
aids = np.concatenate([npz['aids'], npz_test['aids']])
ts = np.concatenate([npz['ts'], npz_test['ts']])
ops = np.concatenate([npz['ops'], npz_test['ops']])

df["start_idx"] = df['total_action'].cumsum().shift(1).fillna(0).astype(int)
df["end_time"] = ts[df["start_idx"] + df["total_action"] - 1]

CPU times: user 6.9 s, sys: 3.82 s, total: 10.7 s
Wall time: 11.6 s


In [3]:
df.head()

,session,total_action,session_start_time,session_end_time,start_idx,end_time
0,0,147,1659304800,1661103727,0,1661103727
1,1,27,1659304800,1660857067,147,1660857067
2,2,13,1659304800,1660577379,174,1660577379
3,3,226,1659304800,1661109666,187,1661109666
4,4,3,1659304800,1659304900,413,1659304900


## 1. Training -- Derive ItemCF similarity Matrix

#### CONSTANTS

In [4]:
## Define constants
PARALLEL = 1024
LOOKBACK_WINDOW = 100   ## only fit the latest LOOKBACK_WINDOW to train the sim matrix
#TOPN = 20
ACTION_WEIGHTS = np.array([1.0, 6.0, 3.0])

#### Section A: Utils Functions 
1. Count Item Total likes: The similary score will be normalized by "Item Total Like Scores". In theory, popular items should have less weight in simiarity score.
2. Trimming function: Helpful managing memoery usage. 
3. Method for normalization: Mostly item total like normalization, and max norm(make all sim score between 0 and 1) of the score. 

In [5]:
# ==================================
# Methods for counting Item Total Likes
# ==================================
@nb.jit(nopython=True)
def getItemTotalLikesNaive(aids, ops, item_total_likes, action_weights):
    """
    Stores the total like score of itemXXX in item_total_likes, based on action_weights parameter. np.array([X, Y, Z])
    """
    for idx, item in enumerate(aids):
        if item not in item_total_likes: 
            item_total_likes[item] = 0
        item_total_likes[item] += action_weights[ops[idx]]   ## TODO: For time decay, consider replace with 1, for iuf keep this. 

# ==================================
# Methods for rank and trim the sim score dict
# ==================================
@nb.jit(nopython = True)
def heap_topk(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure
    """
    dic = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q) for _ in range(len(q))][::-1]
    for i in range(len(res)):
        dic[res[i][1]] = res[i][0]
    
    return dic
   
@nb.jit(nopython = True)
def trim_simMatrix_topk(fullSimMatrix, k = 50):
    """
    trim top k items of each "itemX: {itemY: score1, ...}" pair in fullSimMatrix based on sim scores. 
    """
    for item, item_cnt_dict in fullSimMatrix.items():
        fullSimMatrix[item] = heap_topk(item_cnt_dict, k)

# ==================================
# Methods for score normalization
# ==================================

# @nb.jit(nopython=True)
# def itemTotalLikeNorm(fullSimMatrix, item_total_likes):
#     for aid_1, relations in fullSimMatrix.items():
#         for aid_2, sim_score in relations.items():
#             fullSimMatrix[aid_1][aid_2] = sim_score / (item_total_likes[aid_1] * item_total_likes[aid_2]) ** 0.1  ## TODO: consider 0.1 or other small number
            
@nb.jit(nopython=True)
def maxNormSimMatrix(fullSimMatrix):
    for aid_1, relations in fullSimMatrix.items():
        max_num = -np.inf
        for _, sim_score in relations.items():
            if sim_score > max_num:
                max_num = sim_score
        ## DEGUG use, delete later
        if max_num == 0:
            print(aid_1)
            print(fullSimMatrix[aid_1])
        for aid_2, sim_score in relations.items():
#             if max_num == 0:
#                 max_num += 0.001
            fullSimMatrix[aid_1][aid_2] = sim_score / max_num

#### Section B: Sim Score Computation functions

In [6]:

@nb.jit(nopython=True)
def getSimScoresSingleRow(pairs_this_row, start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode):
    """
    Get the sim scores of items within single session, can be ran in parallel within each batch. 
    """
    max_idx = start_idx + length
    min_idx = max(max_idx - LOOKBACK_WINDOW, start_idx)  
    for i in range(min_idx, max_idx):
        for j in range(i+1, max_idx):
            if ts[j] - ts[i] > 2 * 60 * 60: continue  #TODO: try 2h only
            if aids[i] == aids[j]: continue
            
            if mode == "cosine":
                w_ij = action_weights[ops[j]] 
                w_ji = action_weights[ops[i]] 
            elif mode == "iuf":  ## penalize users that had lots of actions TODO: consider location weight
                if ts[max_idx] - start_time > 24 * 60 * 60:
                    special_factor = 0.8
                else:
                    special_factor = 1.0
                
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                w_ij = special_factor * action_weights[ops[j]] * time_gap_weight * loc_weight / math.log1p(length)
                w_ji = special_factor * action_weights[ops[i]] * time_gap_weight * loc_weight / math.log1p(length)
            elif mode == "time_decay":
                ## calculate some time weights of each item, more weights are given when ts is later. #TODO: try adding (i-j) location weight, exponential weight, 0.5 ** (abs(i-j + 1)), 
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                #time_i = 1 + 0.1 ** ((1662328791-ts[i])/(1662328791-1659304800)) #1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800) #  #(1 - 0.8 *(TEST_END_TS - ts[i]) / TIME_SPAN) ** 0.5 # 0.2~1 #   ## time decay weight for item i 
                #time_j = 1 + 0.1 ** ((1662328791-ts[j])/(1662328791-1659304800))  # 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) # #  #(1 - 0.8 *(TEST_END_TS - ts[j]) / TIME_SPAN) ** 0.5   # 
                time_i = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[i])/(1662328791-1659304800)) - 0.6  )))
                time_j = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[j])/(1662328791-1659304800)) - 0.6  )))
                
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                
                w_ij = action_weights[ops[j]] * loc_weight * time_gap_weight * time_i / math.log1p(length)
                w_ji = action_weights[ops[i]] * loc_weight * time_gap_weight * time_j / math.log1p(length)
                
            pairs_this_row[(aids[i], aids[j])] = w_ij / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1
            pairs_this_row[(aids[j], aids[i])] = w_ji / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1

@nb.jit(nopython=True, parallel=True, cache=True)
def getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, action_weights, item_total_likes, mode="cosine"):
    nrows = len(rows)
    pairs_this_batch = [{(0, 0): 0.0 for _ in range(0)} for _ in range(nrows)]
    ## get the sim scores of each batch in seperate sub dict in pairs_this_batch
    for row_i in nb.prange(nrows):  ## run each row of the batch in parallel
        _, start_idx, length, start_time = rows[row_i]
        getSimScoresSingleRow(pairs_this_batch[row_i], start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode)
    ## merge pairs_this_batch into the fullSimMatrix
    for row_i in range(nrows):
        for (aid1, aid2), score in pairs_this_batch[row_i].items():
            if aid1 not in fullSimMatrix: 
                fullSimMatrix[aid1] = {0: 0.0 for _ in range(0)}
            if aid2 not in fullSimMatrix[aid1]:
                fullSimMatrix[aid1][aid2] = 0.0
            fullSimMatrix[aid1][aid2] += score


#### Section C: Train the similarity matrices
1. Derive the total like score first
2. Train 2 similarity matrices, one using iuf(Inverse User Frequence), the other using time_decay method. 

In [7]:
%%time
## get the Total Like matrix
item_total_likes = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.float64)

getItemTotalLikesNaive(aids, ops, item_total_likes, ACTION_WEIGHTS)

CPU times: user 20.7 s, sys: 856 ms, total: 21.6 s
Wall time: 22.4 s


In [8]:
%%time
simMatrices = {}   ## store a few different similarity matrices using different scoring system, for different prediction type
TRIM_CYCLES = 1000   ## trim full sim matrix every XX batches. 
MODES_TO_TRAIN = ["iuf", "time_decay"]

for mode in MODES_TO_TRAIN:
    ## the nested dict to store full sim matrix, {itemX: {itemY: score, itemZ: score, ...}}
    fullSimMatrix = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
    max_idx = len(df)
    batch_idx = 1  ## compute sim matrix for PARALLEL # of rows per batch, have a total of max_idx/PARALLEL batches.
    for idx in tqdm(range(0, max_idx, PARALLEL)):
        rows = df.iloc[idx: min(idx + PARALLEL, max_idx)][['session', 'start_idx', 'total_action', 'session_start_time']].values
        getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, ACTION_WEIGHTS, item_total_likes, mode=mode)
        batch_idx += 1
        if batch_idx % TRIM_CYCLES == 0:
            print("batch_idx: ", batch_idx)
            trim_simMatrix_topk(fullSimMatrix, 100)
            gc.collect()
            break

    
    ## trim top 50 when the training is complete
    trim_simMatrix_topk(fullSimMatrix, 80)   ## TODO: make this num small enough to reduce time for normalization
    ## max norm of each score
    maxNormSimMatrix(fullSimMatrix)
    
    simMatrices[mode] = fullSimMatrix
    
    del fullSimMatrix
    gc.collect()

  8%|▊         | 998/12079 [02:05<36:13,  5.10it/s] 

batch_idx:  1000


  8%|▊         | 998/12079 [02:44<30:29,  6.06it/s]

batch_idx:  1000


CPU times: user 12min 30s, sys: 5min 43s, total: 18min 14s
Wall time: 5min 56s


In [9]:
## A sanity check
simMatrices["iuf"][1517085]

DictType[int64,float64]<iv=None>({243711: 1.0, 1801351: 0.6420301559334096, 1620477: 0.3205418336285459, 804782: 0.29906089873411057, 760663: 0.29806760860968573, 293222: 0.27718592726596897, 307484: 0.27045098602521467, 807502: 0.26387237977535155, 303302: 0.22713320313972551, 1693043: 0.22420742967890614, 355088: 0.17722081735789424, 1799312: 0.17286524364061515, 899438: 0.1725030172989855, 510211: 0.14385176274047243, 1334074: 0.13439749619481603, 779066: 0.13260267556352995, 527845: 0.11615256735060912, 448706: 0.11398814972170555, 1060697: 0.1109937190198574, 493315: 0.08608320039868826, 596425: 0.07842377422809194, 12725: 0.07664318150644019, 1635429: 0.07586586013132607, 231588: 0.0696293930997437, 1458399: 0.0682591022650933, 307896: 0.06333683757385032, 460942: 0.0574160295826731, 748986: 0.05603382771069549, 1511175: 0.05579452926801223, 102536: 0.048253553003078174, 15594: 0.045834551458824854, 1432934: 0.04521856508329519, 336884: 0.03469253902363178, 266228: 0.031350213770

In [20]:

len(simMatrices["iuf"])

1814440

In [22]:
gc.collect()

11016

## 2. Inference -- Make prediction using the matrices derived from above. 

#### Section D: Utils for inference:
1. Select top items to recommend in re-ranking
2. Compute Real time importance of each action (Not in use currently).

In [23]:
@nb.jit(nopython = True)
def heap_topk_return_list(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure, return a list with top "cap" elements with highest score
    """
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q)[1] for _ in range(len(q))][::-1]
    
    return res

# ===================================

# ===================================


@nb.jit(nopython=True)
def getRealTimeActionWeight(ts_action, ts_start, ts_end, op, seq, length, ACTION_WEIGHTS, TRAIN_START_TS, TEST_END_TS):
    """
    This function returns the real time importance weight of test set session actions
    input: 
        ts_action: ts of the action takes place
        ts_start: start_ts of this session
        ts_end: end_ts of this session
        op: type of the action
        seq: seq order this action
        length: total # of actions of this session. 
    """
#     overall_time_span = TEST_END_TS - TRAIN_START_TS
#     #session_time_span = ts_end - ts_start
#     time_to_end_of_session = ts_end - ts_action
#     if time_to_end_of_session > 24 * 60 * 60:
#         time_weight = 0.3
#     else:
#         time_weight = max(2**(1 - time_to_end_of_session/3600) - 1, 0.4)
    ## TODO: add time decay 
    action_weight = ACTION_WEIGHTS[op]
    sequence_weight = max(2 ** (seq/length) - 1, 0.5) ## floor at 0.1  #np.power(2, np.linspace(seq_weight_const, 1, length))[::-1] - 1
    res = action_weight * sequence_weight # * time_weight
    
    return res

#### Section E: Main Logic in Making Inferences
1. clicks_inferences: time_decay sim matrix + regular action weights <1, 6, 3>.
2. carts_inferencs: iuf sim matrix + weights <4, 2, 5> (as clicks actions tend to lead to cart action next).
3. orders_inferences: iuf sim matrix + regular action weights <1, 6, 3>.

In [79]:
@nb.jit(nopython=True)
def inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    ending_idx = starting_idx + length
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]
    
    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)

    ## TODO: Add info_data list to carry all the features, <prev_int, seq_weight, time_weight, >
    feat = []
    

    ## get unique aid of each session 
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    for a in candidates:
        unique_aids[a] = 0
    
    if len(unique_aids) >= 20:   ## if the user has many actions, recommend based on the itemes it had been interacted with only.
        PREV_INTERACT_BONUS = 10
        sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
        for aid, op, w in zip(candidates, candidates_ops, sequence_weight):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            potential_to_recommend[aid] += w * test_ops_weights[op] * PREV_INTERACT_BONUS 
        #result[session] = np.array(heap_topk_return_list(potential_to_recommend, 20)) 
    else:  
        for idx in range(starting_idx, ending_idx):
            candidate = aids[idx]
            if candidate not in potential_to_recommend:
                potential_to_recommend[candidate] = np.inf ## ensure large weights on items had interacted with. 
    
    ## In case some items are duplicates, when potential_to_recommend not yet reach 20, impute with items from sim matrix
    if len(potential_to_recommend) < 80: ## CAUTIOUS: validation purpose only 
        sequence_weight = np.power(2, np.linspace(0.1, 1, len(candidates))) - 1   ## CHANGE_MADE: 0.3 -> 0.1
        for idx in range(starting_idx, ending_idx):
            candidate = aids[idx] 
            time_weight = 1 + 0.1 ** ((1662328791-ts[idx])/(1662328791-1659304800))    ## TODO: consider 
            candidate_realtime_weight = test_ops_weights[ops[idx]] * sequence_weight[idx-starting_idx] * time_weight  
            ## load the potential items to recommend,
            if candidate not in full_sim_matrix: 
                continue
            for similar_item in full_sim_matrix[candidate]:
#                 if similar_item in candidates:    ## skip the item if the it's already been interacted
#                     continue
                if similar_item not in potential_to_recommend:
                    potential_to_recommend[similar_item] = 0
                potential_to_recommend[similar_item] += full_sim_matrix[candidate][similar_item] * candidate_realtime_weight 
    
    result[session] = np.array(heap_topk_return_list(potential_to_recommend, 80))   ## CAUTIOUS: validation purpose only
    
@nb.jit(nopython=True)
def run_inference_parallel(rows, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights)

In [80]:
%%time
result_iuf = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

result_iuf_2 = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

result_time_decay = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
    run_inference_parallel(rows, aids, ops, ts, result_iuf, simMatrices["iuf"], np.array([2.0, 6.0, 6.0]))
    run_inference_parallel(rows, aids, ops, ts, result_iuf_2, simMatrices["iuf"], np.array([4.0, 2.0, 5.0]))   ## considebly add the weights for click action in the real time.
    run_inference_parallel(rows, aids, ops, ts, result_time_decay, simMatrices["time_decay"], np.array([3.0, 6.0, 3.0]))

100%|██████████| 1742/1742 [08:22<00:00,  3.47it/s]

CPU times: user 6min 34s, sys: 53.8 s, total: 7min 28s
Wall time: 8min 23s


In [81]:
result_time_decay[11098528]

array([  11830, 1732105,  588923,  571762,  884502,  876129, 1157882,
       1182614,  307904, 1790438,  231487, 1517680, 1633746,   77440,
       1586171,  855613,  735729,  205357,  756588,  523174,  258814,
        636101,  322370, 1689044,  532616, 1317291, 1718231, 1609734,
        409620, 1125638,  603583,  215561, 1052212, 1394029,  822934,
       1853703, 1383529, 1425172,  135833,  500334, 1519088,   19468,
       1349230, 1390152,  542780, 1667019,  490677,  448755, 1169267,
       1428162, 1695994,   42241,  804966, 1677053, 1311701,  672942,
        487136, 1197172,   45494,  697336, 1572401,  432989, 1446918,
        577040,  696526, 1241036, 1123537, 1307712, 1041771,  600258,
       1307461,  293000,  460148, 1032776,   87442,  198496,  972466,
       1421968, 1097576,  823991])

In [82]:
gc.collect()

37988

### Section F: Save the features

In [ ]:
@nb.jit(nopython=True)
def save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    PREV_INTERACT_BONUS = 10
    NEARBY_ACTION_BONUS = 1.5
    
    ending_idx = starting_idx + length
    end_time = ts[ending_idx]
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]

    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)

    ## item_features
    item_features = []

    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    for a in candidates:
        if a not in unique_aids.keys():
            unique_aids[a] = 0
        unique_aids[a] += 1

    if len(unique_aids) >= 20:
        
        sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
        time_weights = []
        for idx in range(starting_idx, ending_idx):
            if end_time - ts[idx] < 2 * 60 * 60:   ## apply nearby action bonus
                time_weight = (1 + 0.5 ** ((end_time - ts[idx])/(end_time - start_time))) * NEARBY_ACTION_BONUS
            else:
                time_weight = 1 + 0.5 ** ((end_time - ts[idx])/(end_time - start_time))
            time_weights.append(time_weight)
        time_weights = time_weights[::-1]

        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            potential_to_recommend[aid] += seq_w * time_w * test_ops_weights[op] * PREV_INTERACT_BONUS
            item_features.append(seq_w, time_w, 1)
            

   
    
@nb.jit(nopython=True)
def save_feature_parallel(rows, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights)

## 3. Submissions

In [83]:
%%time
subs = []
op_names = ["clicks", "carts", "orders"]

for result, op in zip([result_time_decay, result_iuf_2, result_iuf], op_names):
    sub = pd.DataFrame({"session_type": result.keys(), "labels": result.values()})
    sub.session_type = sub.session_type.astype(str) + f"_{op}"
    sub.labels = sub.labels.apply(lambda x: " ".join(x.astype(str)))
    subs.append(sub)
    
submission = pd.concat(subs).reset_index(drop=True)
#sub.sort_values(by=["session_type"])  ## optional
#submission.to_csv('submission.csv', index = False)
submission

CPU times: user 5min 33s, sys: 18.9 s, total: 5min 52s
Wall time: 6min 12s


,session_type,labels
0,11098528_clicks,11830 1732105 588923 571762 884502 876129 1157...
1,11098529_clicks,1105029 1632356 1049489 612829 295362 333991 1...
2,11098530_clicks,409236 264500 1603001 583026 963957 254154 752...
3,11098531_clicks,1728212 1557766 1553691 1449555 1365569 130963...
4,11098532_clicks,876469 7651 1202618 1159379 77906 476681 17040...
...,...,...
5351206,12899774_orders,33035 1539309 819288 771913 31490 95488 218795...
5351207,12899775_orders,1743151 1760714 1163166 1255910 1498443 783827...
5351208,12899776_orders,548599 695829 1737908 773354 1762353 1440959 9...
5351209,12899777_orders,384045 1308634 1688215 395762 703474 1486067 2...


In [85]:
%%time
submission.to_csv('../../allData/validationData/p_v_579_80_items.csv', index = False)

CPU times: user 47 s, sys: 3.32 s, total: 50.3 s
Wall time: 50.9 s


In [84]:
len(submission.iloc[0]["labels"].split(" "))

80

In [40]:
len(df_test)

1783737

In [71]:
gc.collect()

60